In [1]:
import sys
sys.path.insert(0,"/Library/Python/2.7/site-packages")
#sys.path

import numpy as np
import scipy as sp
import pandas as pd
#print np.__version__
#print np.__file__

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as ptch
from matplotlib import rcParams
#from matplotlib.dates import YearLocator, MonthLocator, DayLocator, DateFormatter
import matplotlib.dates as mdates
import matplotlib.collections as mplcoll
import matplotlib.colors as mplcolor
#print mpl.__version__
#print mpl.__file__

import json
from operator import itemgetter
%matplotlib inline

import datetime as DT

import MySQLdb

##  Load Generic Functions

In [2]:
%run 'new_user_activation_funcs.ipynb'

## Load Data

In [3]:

#thisDF = grabDF_new_users_daily("2016-01-01", "2016-07-01")
thisDF = grabDF_new_users_daily("2016-01-01", "2016-06-06")

startday = thisDF.iloc[0].day
print startday
thislen = len(thisDF.index)
print thislen
#thisDF['newday'] = pd.date_range(start=startday,periods=thislen,freq='D')
#thisDF.index = pd.date_range(start=startday,periods=thislen,freq='D')

#thisDF.set_index('day')

#
# this was me trying to play with date stuff to fix below error ... but to no avail ....
#
#newDF = thisDF.set_index('day')
#newDF = thisDF.set_index('newday')
#newDF.index.inferred_freq = 'D'
#newDF.index.freq = 'D'
#print newDF.head()
#print newDF.dtypes
#print newDF.index
#print newDF.index.freq

grabbed new users from:  2016-01-01  to  2016-06-06
2016-01-01 00:00:00
158


In [4]:
import numpy as np  
from scipy.stats import norm, mstats

def mk_test(x, alpha = 0.05):  
    """   
    Input:
        x:   a vector of data
        alpha: significance level (0.05 default)

    Output:
        trend: tells the trend (increasing, decreasing or no trend)
        h: True (if trend is present) or False (if trend is absence)
        p: p value of the significance test
        z: normalized test statistics 

    Examples
    --------
      >>> x = np.random.rand(100)
      >>> trend,h,p,z = mk_test(x,0.05) 
    """
    n = len(x)
    #print "n=", n

    # calculate S 
    s = 0
    for k in range(n-1):
        for j in range(k+1,n):
            s += np.sign(x[j] - x[k])
            
    # calculate the unique data
    unique_x = np.unique(x)
    g = len(unique_x)

    # calculate the var(s)
    if n == g: # there is no tie
        var_s = (n*(n-1)*(2*n+5))/18
    else: # there are some ties in data
        tp = np.zeros(unique_x.shape)
        for i in range(len(unique_x)):
            tp[i] = sum(unique_x[i] == x)
        var_s = (n*(n-1)*(2*n+5) + np.sum(tp*(tp-1)*(2*tp+5)))/18

    #print "s=", s
    if s>0:
        z = (s - 1)/np.sqrt(var_s)
    elif s == 0:
        z = 0
    elif s<0:
        z = (s + 1)/np.sqrt(var_s)

    #print "z=", z
    # calculate the p_value
    p = 2*(1-norm.cdf(abs(z))) # two tail test
    h = abs(z) > norm.ppf(1-alpha/2) 

    if (z<0) and h:
        trend = 'decreasing'
    elif (z>0) and h:
        trend = 'increasing'
    else:
        trend = 'no trend'

    return trend, h, p, z

In [5]:
#trend.dropna(inplace=True)
#cleantrend = trend[~np.isnan(trend)]
#print trend
#print cleantrend

cleantrend = thisDF['ExpMA7']

test_trend,h,p,z = mk_test(cleantrend, alpha=0.05)  
print test_trend, h  
print z, p  

decreasing True
-16.971596375 0.0
